## Mapping the NSD to the FSL for the creation of a SWAT compatiable usersoil dataset

this colleciton of scripts takes in the NSD .csv dataset, and the SoilKsatDB database to produce / derive and complete values using a Yggdrasil Decision Forest regresion model to fill in blanks.

While the NSD data can be viewed freely via the NSD explorer at https://viewer-nsdr.landcareresearch.co.nz/search, this ommits some key information such as the bulk density and a number of other key values that are essential in the modeling process,
as such to replicate this you'll need to contact Manaaki Whenua directly for access to the raw NSD.

Many of the key points in this process are derived from the process outlined by Parshotam, 2018 ( https://environment.govt.nz/assets/OIA/Files/20-D-02513_0.pdf ).
This includes the mapping for missing NZSC codes found in the FSL but not the NSD, as well as the site selection used in mapping the NZSC codes to NSD samples, aswell as helping in formulate much of the underlying process and logic.

After collating and cleaning the bulk of the NSD data, Regression models are derived using the radnom forest models provided by the YDF python package ( https://ydf.readthedocs.io/en/stable/#next-steps ), which builds on from the deprecited tensor flow random forest model. 

Missing fine earth values are predicted as well as missing rock percentages, follwoing this, the bulk density and avaliable water capacity are predicted, using data from within the NSD.

Saturated hyrdaulic conductivity is missing from the NSD, as such the external dataset SoilKsatDB from Gupta, S. et al, 2021 ( https://doi.org/10.5194/essd-13-1593-2021 ) is used to fill in these gaps, by building a similar Regression model based on this data set, it can be applied to the data present in the NSD, initally I inteded to use the equations outlined by Saxton and Rawls, 2006, for the SPAW tool, however given their method I assumed basing it off a more complex dataset thats been made avaliable since would produce better results.

Due to limited time, there has been limited analysis and in depth evauluation on the results from these predictions, however especially for the fine earth percentages the derived models had quite low residual error, but for the more complex values such as saturated hydraulic conducitivity and avaliable water capacity, the models produced might not be adequate, however bulk density was on par with the fine earth models. 


## References

- Parshotam, 2018. New Zealand SWAT: Deriving a New Zealand-wide soils dataset from the NZ-NSD for use in the Soil and Water Assessment Tool (SWAT). Report prepared for Ministry for the Enviroment. Project: WL18033. Aqualinc Research Limited. https://environment.govt.nz/assets/OIA/Files/20-D-02513_0.pdf

- Manaaki Whenua - Landcare Research 2020. National Soils Database (NSD). https://doi.org/10.26060/95m4-cz25

- Gupta, S., Hengl, T., Lehmann, P., Bonetti, S., and Or, D.: SoilKsatDB: global database of soil saturated hydraulic conductivity measurements for geoscience applications, Earth Syst. Sci. Data, 13, 1593–1612, https://doi.org/10.5194/essd-13-1593-2021, 2021. 

- https://ydf.readthedocs.io/en/stable/#next-steps

- Saxton, K. E., and W. J. Rawls. “Soil Water Characteristic Estimates by Texture and Organic Matter for Hydrologic Solutions.” Soil Science Society of America Journal, vol. 70, no. 5, 2006, p. 1569, https://doi.org/10.2136/sssaj2005.0117.

## Reading & Preprocessing the NSD data & Gloabl Ksat data

this uses the scripts and classes defined in the Useable_NSD_dataset_preprocessing python file. there is alot of choices made in this section any major adjustments to my methods would likely involve changing this section

the SoilKsatDB is processed by the Global_kSat_preprocessing script. 

In [45]:
import pandas as pd

import sys
sys.path.append("Scripts")

## Adding the scripts path to enable calling the scripts from within the "scripts" folder
## sd_horizon_data and sd_horizon both have very similar data, however each is missing one or two key columns, so I've decided to use both
hrzn_1 = "data/sd_horizon_data.csv"
hrzn_2 = "data/sd_horizon.csv"

## This is the bulk of the NSD's data, and contains all the actual measurements for each horizon
obs_1 = "data/ob_observation_data.csv"

## Generic descirptions of the soil sites, just used for the gentic description column
soil = "data/sd_soil.csv"


In [46]:

from NSD_dataset_preprocessing import Read_NSD_data

## Read_NSD_data is a large function which takes in the provided datasets and cleans, converts and maps all the data into a single table which makes all subsequent steps easier.
## a non insignficant amount of the run time is simply loading the ob_observation data

NSD_data = Read_NSD_data(hrzn_1, hrzn_2, obs_1, soil)

NSD_data.to_csv("intermediate_data/NSD_processed_data.csv", index=False)


## The most important variable for soil mapping is total carbon percent.
## Filter out rows with missing total carbon percent values.
Useable_NSD_data = NSD_data.dropna(subset=["total_carbon_percent"])

## There are some negative values in horzion depth, which descrbie leaf liter and top vegetation layers,
#  this data is kinda weird to work with and causes problems.

Useable_NSD_data = Useable_NSD_data[Useable_NSD_data["horizondepth_maxval"].astype(float) > 0]

Useable_NSD_data.to_csv("intermediate_data/Useable_NSD_data.csv", index=False)

In [47]:
from NSD_dataset_preprocessing import ConvertNSD_To_ML_readable

## Takes the dataset produced by the previous function and does numerous operations to split and otherwise modify most of the data columns into fields and binary flags which can be used by a machine learning model.
Predictor_data = ConvertNSD_To_ML_readable(Useable_NSD_data)

Predictor_data.to_csv("intermediate_data/ML_readable_NSD_data.csv", index=False)

/home/luka/Documents/SWAT/Soil_mapping_refinement/fsl_mapping_org/Scripts/NSD_dataset_preprocessing.py:506: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df["CLR_matrix_hue"] = df["CLR_matrix_hue"].replace({"NAN":"UNK"})


In [48]:
from Global_kSAT_preprocessing import preprocess_ksat_data

## Read in the SoilKsatDB data
sol_ksat_df = pd.read_csv("data/sol_ksat.pnts_horizons.csv", compression="gzip")

## the SoilKsatDB dataset has a handful of super high values, which cuased issues with the model, as such these have been clipped off to give a more generally accurate model.
sol_ksat_trainnig_data = preprocess_ksat_data(sol_ksat_df, outlier_threshold=2000)

sol_ksat_trainnig_data.to_csv("intermediate_data/sol_ksat_trainingdata.csv")



529.5228582448085
220.6345242686702


/home/luka/Documents/SWAT/Soil_mapping_refinement/fsl_mapping_org/Scripts/Global_kSAT_preprocessing.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sol_ksat_training_set["awc"] = (sol_ksat_training_set["w3cld"] - sol_ksat_training_set["w15l2"])
/home/luka/Documents/SWAT/Soil_mapping_refinement/fsl_mapping_org/Scripts/Global_kSAT_preprocessing.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sol_ksat_training_set["ksat_lab"] = ( sol_ksat_training_set["ksat_lab"] * 10 ) / 24.0


## Predicting NSD Data

applies the YDF random forest model onto the dataset created in the previous step.

In [49]:
from NSD_Predictor import Predict_data

## based on the NSD and SoilKsatDB data that has been preprocessed
# this returns the data in the machine learning readable format.
NSD_predicted = Predict_data(Predictor_data, sol_ksat_trainnig_data)

NSD_predicted.to_csv("intermediate_data/NSD_Horizions_Predicted.csv")

## Fill in the missing values of the columns of interest in the Usable NSD dataset.


Useable_NSD_data["clay_percent"] = Useable_NSD_data["clay_percent"].fillna(NSD_predicted["clay_percent"])
Useable_NSD_data["silt_percent"] = Useable_NSD_data['silt_percent'].fillna(NSD_predicted["silt_percent"])
Useable_NSD_data["sand_percent"] = Useable_NSD_data['sand_percent'].fillna(NSD_predicted["sand_percent"])

Useable_NSD_data["rock_fragment_percent"] = Useable_NSD_data['rock_fragment_percent'].fillna(NSD_predicted["rock_fragment_percent"])

Useable_NSD_data["whole_bulk_density"] = Useable_NSD_data['whole_bulk_density'].fillna(NSD_predicted["whole_bulk_density"])
Useable_NSD_data["available_water_capacity"] = Useable_NSD_data['available_water_capacity'].fillna(NSD_predicted["available_water_capacity"])

Useable_NSD_data["ksat"] = NSD_predicted["ksat"]

Useable_NSD_data.to_csv("intermediate_data/NSD_Horizions_Usable_filled.csv")

Feature DESG_suffix_E is a NUMERICAL feature with all values recorded in the data spec set to the same value. The feature will likely not be useful during model training.
Feature DESG_suffix_M is a NUMERICAL feature with all values recorded in the data spec set to the same value. The feature will likely not be useful during model training.
Feature DESG_suffix_q is a NUMERICAL feature with all values recorded in the data spec set to the same value. The feature will likely not be useful during model training.
Feature DESG_suffix_F is a NUMERICAL feature with all values recorded in the data spec set to the same value. The feature will likely not be useful during model training.
Train model on 3989 examples
Model trained in 0:00:01.686344
Feature DESG_suffix_E is a NUMERICAL feature with all values recorded in the data spec set to the same value. The feature will likely not be useful during model training.
Feature DESG_suffix_X is a NUMERICAL feature with all values recorded in the data sp

## Map the Predicted NSD results the FSL 
this step uses the tables derived by Parshotam, 2018

This is the last step in the process and maps the completed / predicted NSD data, to a corresponding FSL mapping, as is this makes a national scale Soil map based on unique NZSC codes

This process could be adjusted to make a site specific mapping, if more apporiate NSD samples are identified for a specific site, 

FSL layer has a lot of assumptions and a national scale dataset needs to make some consecisions 



In [50]:

import geopandas as gd

fsl_shape_file = r"lris-fsl-north-island-v11-all-attributes-SHP/fsl-north-island-v11-all-attributes.shp"

## These are codes that exist in FSL but not in NSD,
# so we need to map them to the closest equivalent, this was derived in the following paper https://environment.govt.nz/assets/OIA/Files/20-D-02513_0.pdf

missing_code_lookup = pd.read_csv("data/MissingNZSC_Replacements.csv")

## This is the table that associates each NZSC code, with a NSD site, this is based again on the work done here https://environment.govt.nz/assets/OIA/Files/20-D-02513_0.pdf
## Because this process uses the NZSC code to match sites, any adjustment to this process, 
# say if muitple sites want to be used for slightly different FSL regions which have matching NZSC codes, will need a change in the following logic

nzsc_nsd_lookup_df = pd.read_csv("data/FSL_NZSC_NSD_SITE_MAP.csv")

## Split the NZSC codes and grab the short hand version
Useable_NSD_data["classifier_nzsc_short"] = Useable_NSD_data["classifier_nzsc"].str.split().str[0]
nsd_nzsc_codes = Useable_NSD_data["classifier_nzsc_short"].unique().tolist()

## Read in the FSL layer, apply the missing NZSC code map, and map in the assigned NSD site codes.
## This could also use the Main soil type field as a mapping, however this will need a unique NSD Site code map.

fsl_data = gd.read_file(fsl_shape_file)

fsl_data["NSD_NZSC_CODE_MAP"] = fsl_data["DOMNZSC"].apply(lambda x: x if x in nsd_nzsc_codes else (missing_code_lookup[x] if x in missing_code_lookup else "MISSING"))
fsl_data["NSD_SITE_CODE"] = fsl_data["NSD_NZSC_CODE_MAP"].apply(lambda x: nzsc_nsd_lookup_df.loc[nzsc_nsd_lookup_df["NZSC_CODE"] == x, "NSD_SITE_CODE"].values[0] if x in nzsc_nsd_lookup_df["NZSC_CODE"].values else "MISSING")

## find the NSD layers that are used in the site lookup table
sig_nsd_layers = Useable_NSD_data[Useable_NSD_data["site_identifier"].isin(nzsc_nsd_lookup_df["NSD_SITE_CODE"].unique().tolist())]


In [ ]:
## Create the Headers for the usersoil dataset. 

## Find the number of sites, and the maximum numnber of layers of any of them.
SITE_NLAYERS = sig_nsd_layers["site_identifier"].value_counts()
# MAX_LAYERS = int(sig_nsd_layers["horizonnumber"].astype(float).max())
MAX_LAYERS = 10
print(MAX_LAYERS)
## Two sets of headers, one which is per site, and one which is per layer per site.

primary_headers = ["OBJECTID","SITE_ID_SB","SITE_ID","MUID","SEQN","SNAM","S5ID","CMPPCT","NLAYERS","HYDGRP","SOL_ZMX","ANION_EXCL","SOL_CRK","TEXTURE"]
layer_headers = ["LAYER_ID#", "SOL_Z#", "SOL_BD#", "SOL_AWC#", "SOL_K#", "SOL_CBN#", "CLAY#", "SILT#", "SAND#", "ROCK#", "SOL_ALB#","USLE_K#","SOL_EC#"]

addon_headers = ["SOL_CAL#", "SOL_PH#"] ## Not activated but these columns still need to be in the usersoil file, otherwise it causes issues.
## Given the maximum number of layers, populate the layer headers acordingly.

layer_headers_renamed = [header.replace("#", str(i)) for i in range(1, MAX_LAYERS+1) for header in layer_headers]
addon_headers = [header.replace("#", str(i)) for i in range(1, MAX_LAYERS+1) for header in addon_headers] ## Not activated but these columns still need to be in the usersoil file, otherwise it causes issues.

## make the data frame

usersoil_headers = primary_headers + layer_headers_renamed + addon_headers
usersoil = pd.DataFrame(columns=usersoil_headers, index=range(len(SITE_NLAYERS)))




10


In [52]:
## Map the Per Site Columns to target NSD sites ( uses the NSD_data data not the sa_site data )

## Need to convert the horizon number form a string float to a interger to prevent issues later on ( there will be a b)
sig_nsd_layers["horizonnumber"] = sig_nsd_layers["horizonnumber"].astype(float).astype(int)

usersoil["SITE_ID"] = sig_nsd_layers["sd_soil_id"].unique().tolist()  

sig_ob_site_id = sig_nsd_layers[["sd_soil_id", "site_identifier"]].set_index("sd_soil_id").to_dict()["site_identifier"]
sig_ob_nlayers = sig_nsd_layers[["sd_soil_id", "horizonnumber"]].groupby("sd_soil_id").max().to_dict()["horizonnumber"]
sig_ob_class = sig_nsd_layers[["sd_soil_id", "classifier_nzsc"]].set_index("sd_soil_id").to_dict()["classifier_nzsc"]
# sig_ob_max_depth = sig_nsd_layers[["sd_soil_id", "horizondepth_maxval"]].groupby("sd_soil_id").max().to_dict()["horizondepth_maxval"]


usersoil["SITE_ID_SB"] = usersoil["SITE_ID"].map(sig_ob_site_id)
usersoil["NLAYERS"] = (usersoil["SITE_ID"].map(sig_ob_nlayers).astype(float)).astype(int)

## The QSWAT plugin I'm using for SWAT cannot handle more than 10 layers, while not common, some sites contain more than 10, in one case the best / only choice has 13 layers,
    # clipping off at 10 is a crude solution and potentially "disolving" some layers based on comminality would probably be better, but this works.
usersoil["NLAYERS"] = usersoil["NLAYERS"].apply(lambda x: x if x <= 10 else 10)
print(usersoil["NLAYERS"].value_counts())

## Sets the SNAM as the NZSC code and strips off the long hand name
usersoil["SNAM"] = usersoil["SITE_ID"].map(sig_ob_class)
usersoil["SNAM"] = usersoil["SNAM"].str.split().str[0]



## Gets the max depth and converts it to mm
# usersoil["SOL_ZMX"] = usersoil["SITE_ID"].map(sig_ob_max_depth)
# usersoil["SOL_ZMX"] = usersoil["SOL_ZMX"].astype(float) * 10

usersoil["OBJECTID"] = range(1, len(usersoil)+1)

usersoil.to_csv("nzsc_usersoil.csv")

NLAYERS
5     44
7     29
6     22
4     18
8     12
3     11
10     7
9      4
2      1
Name: count, dtype: int64


/tmp/ipykernel_28259/3287003887.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_nsd_layers["horizonnumber"] = sig_nsd_layers["horizonnumber"].astype(float).astype(int)


In [53]:
## I've run into far to many issues in this section given what it's doing. 
# if it doesn't work try restarting the kernel and running again ( it's solved too many of the issues )


from calc_usle_k import calc_usle_k
import numpy as np

## Create a dict of all the horizion ids, based on the site id and horizion number
sig_ob_layer_ids = sig_nsd_layers[["sd_horizon_id", "horizonnumber", "sd_soil_id"]].set_index(["sd_soil_id", "horizonnumber"]).to_dict()["sd_horizon_id"]
# print(sig_ob_layer_ids)
## Becuase of the preprocessing steps, this doesn't need to handle any errors in the data, such as soil layers above depth 0, however this should be kept in mind if issues arise.

##  Its not best practice to iterate over pandas rows, but this isn't a high frequency operation.

## Becuase, there are often layers, either above ground ( leaf litter etc ), that I've ommited, the horizonnumber value, doesn't always line up with what is present
## This accounts for that and adjusts the data accordingly before mapping the actual values in.
for row in usersoil.itertuples():
    site_id = row.SITE_ID
    missing_layers = 0
    i = 1
    
    while i < row.NLAYERS + 1:
        if (site_id, i) in sig_ob_layer_ids:
            usersoil.at[row.Index, f"LAYER_ID{i-missing_layers}"] = sig_ob_layer_ids.get((site_id, i), None)
            # print(f"Mapping layer {i} for site {site_id} to horizon id {usersoil.at[row.Index, f'LAYER_ID{i}']}, for Layer {i-missing_layers}")
            i += 1
        else:
            missing_layers += 1
            i += 1
            # print(f"Skipping missing or shallow layer {i} for site {site_id}")
    
    usersoil.at[row.Index, "NLAYERS"] = row.NLAYERS - missing_layers
    

## Maps the data both predicted and measured from the NSD into the usersoil data    
for row in usersoil.itertuples():
    
    site_id = row.SITE_ID
    
    for i in range(1, row.NLAYERS + 1):
        layer_id = getattr(row, f"LAYER_ID{i}")
        # print(layer_id)
        if pd.notna(layer_id):
            
            ## Grab the row based on the horizon id, and reset the index so it can be accessed via .loc[0]            
            target_row = sig_nsd_layers.loc[sig_nsd_layers['sd_horizon_id'] == layer_id].reset_index()
            
            rock_frag = float(target_row['rock_fragment_percent'].loc[0])           
            clay = float(target_row["clay_percent"].loc[0])
            silt = float(target_row["silt_percent"].loc[0])
            sand = float(target_row["sand_percent"].loc[0])
            
            ### Removes any errors in the fine earth values, as to ensure they sum to 100%
            fine_earth_frac = (clay+silt+sand)/100.0
            clay = clay / fine_earth_frac
            silt = silt / fine_earth_frac
            sand = sand / fine_earth_frac
            
            ## The SWAT process needs the rock frag and fine earth to sum to 100, this adjusts the fine earth to take into account the rock percentage
            rock_frag_frac = 1 - (rock_frag)/100
            clay = rock_frag_frac * clay
            silt = rock_frag_frac * silt
            sand = rock_frag_frac * sand
            
            ## Write the commupted values into the usersoil at the give column and row
            usersoil.at[row.Index, f"SOL_Z{i}"] = float(target_row['horizondepth_maxval'].loc[0]) * 10
            if i == row.NLAYERS:
                usersoil.at[row.Index, "SOL_ZMX"] = float(target_row['horizondepth_maxval'].loc[0]) * 10
            
            usersoil.at[row.Index, f"ROCK{i}"] = rock_frag
            usersoil.at[row.Index, f"CLAY{i}"] = clay
            usersoil.at[row.Index, f"SILT{i}"] = silt
            usersoil.at[row.Index, f"SAND{i}"] = sand
                        
            usersoil.at[row.Index, f"SOL_BD{i}"] = float(target_row["whole_bulk_density"].loc[0])
            usersoil.at[row.Index, f"SOL_AWC{i}"] = float(target_row["available_water_capacity"].loc[0])
            usersoil.at[row.Index, f"SOL_CBN{i}"] = float(target_row["total_carbon_percent"].loc[0])
            usersoil.at[row.Index, f"SOL_K{i}"] = abs(float(target_row["ksat"].loc[0]))     ## This might be a bad idea, but negative Ksat values don't make sense.

            usersoil.at[row.Index, f"SOL_ALB{i}"] = float(target_row['CLR_alb'].loc[0])

            ## Calculate the Usle K value
            usersoil.at[row.Index, f"USLE_K{i}"] = calc_usle_k(usersoil.at[row.Index, f"SOL_CBN{i}"], usersoil.at[row.Index, f"SAND{i}"], usersoil.at[row.Index, f"SILT{i}"], usersoil.at[row.Index, f"CLAY{i}"])
            

## Add a blank row at the start for water values
usersoil.loc[len(usersoil)] = None
usersoil = usersoil.shift()
usersoil.at[0, "SNAM"] = "WTR"
 
## Fill in some of the unused columns with default values
usersoil["HYDGRP"] = usersoil["HYDGRP"].fillna("C")         ## Not required, filled with C as a placeholder
usersoil["ANION_EXCL"] = usersoil["ANION_EXCL"].fillna(0.5)
usersoil["SOL_CRK"] = usersoil["SOL_CRK"].fillna(0.5)
usersoil["TEXTURE"] = usersoil["TEXTURE"].fillna("UNK")       ## Not required 

usersoil = usersoil.fillna(0)

### Drop the columns containg the various id's, doesn't drop the SB site identifier code tho, so each layer can still be looked up in the NSD for validation
usersoil = usersoil[usersoil.columns.drop(list(usersoil.filter(regex='LAYER_ID')))]
usersoil = usersoil[usersoil.columns.drop("SITE_ID")]


usersoil = usersoil.drop_duplicates(subset="SNAM")

usersoil.to_csv("nzsc_usersoil.csv")

/tmp/ipykernel_28259/2039535475.py:93: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  usersoil["ANION_EXCL"] = usersoil["ANION_EXCL"].fillna(0.5)
/tmp/ipykernel_28259/2039535475.py:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  usersoil["SOL_CRK"] = usersoil["SOL_CRK"].fillna(0.5)
/tmp/ipykernel_28259/2039535475.py:97: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_s